In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import f1_score


In [27]:
# 분류 모델 정의
class CombinedClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(CombinedClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

def load_data(file_path, feature_path):
    df = pd.read_csv(file_path)
    documents = df['cleaned_text'].tolist()
    labels = df['cEXT'].apply(lambda x: 1 if x == 'y' else 0).tolist()
    features = np.load(feature_path)
    return documents, labels, features

def build_dataloader(X, y, batch_size):
    tensor_x = torch.tensor(X).float()  
    tensor_y = torch.tensor(y).long()
    dataset = TensorDataset(tensor_x, tensor_y) 
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    return dataloader


def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss, total_accuracy, all_predictions, all_targets = 0, 0, [], []
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, pred = torch.max(output, dim=1)
        all_predictions.extend(pred.cpu().numpy())
        all_targets.extend(target.cpu().numpy())
        accuracy = torch.sum(pred == target).item() / len(target)
        total_accuracy += accuracy
    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_accuracy / len(train_loader)
    avg_f1 = f1_score(all_targets, all_predictions, average='macro')
    return avg_loss, avg_accuracy, avg_f1

def test(model, test_loader, criterion, device):
    model.eval()
    total_loss, total_accuracy, all_predictions, all_targets = 0, 0, [], []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            _, pred = torch.max(output, dim=1)
            all_predictions.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            accuracy = torch.sum(pred == target).item() / len(target)
            total_accuracy += accuracy
    avg_loss = total_loss / len(test_loader)
    avg_accuracy = total_accuracy / len(test_loader)
    avg_f1 = f1_score(all_targets, all_predictions, average='macro')
    return avg_loss, avg_accuracy, avg_f1


def run_training(file_path, feature_path, batch_size, input_size, hidden_size, num_classes, lr, epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {device} device.")

    _, labels, features = load_data(file_path, feature_path)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)
    
    train_loader = build_dataloader(X_train, y_train, batch_size)
    test_loader = build_dataloader(X_test, y_test, batch_size)

    model = CombinedClassifier(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        train_loss, train_acc, train_f1 = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc, test_f1 = test(model, test_loader, criterion, device)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, Test F1: {test_f1:.4f}")

file_path = "C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\전처리데이터\\BigFive\\BigFive_prepro_sen.csv"
feature_path = "C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\전처리데이터\\BigFive\\concat_cls.npy"
batch_size = 16
input_size = 768 + 81  # Assuming BERT [CLS] token output size + LIWC feature size
hidden_size = 768 + 81  # Example hidden size, adjust as needed
num_classes = 2
lr = 2e-4
epochs = 20

In [28]:
run_training(file_path, feature_path, batch_size, input_size, hidden_size, num_classes, lr, epochs)

Using cpu device.
Epoch 1/20 - Train Loss: 0.7096, Accuracy: 0.5154, Test Loss: 0.6928, Accuracy: 0.5441, Test F1: 0.4306
Epoch 2/20 - Train Loss: 0.6914, Accuracy: 0.5513, Test Loss: 0.7646, Accuracy: 0.4865, Test F1: 0.3338
Epoch 3/20 - Train Loss: 0.6815, Accuracy: 0.5694, Test Loss: 0.6968, Accuracy: 0.5386, Test F1: 0.4135
Epoch 4/20 - Train Loss: 0.6678, Accuracy: 0.5917, Test Loss: 0.6821, Accuracy: 0.5746, Test F1: 0.5265
Epoch 5/20 - Train Loss: 0.6595, Accuracy: 0.6112, Test Loss: 0.7004, Accuracy: 0.5187, Test F1: 0.4721
Epoch 6/20 - Train Loss: 0.6472, Accuracy: 0.6454, Test Loss: 0.6937, Accuracy: 0.5282, Test F1: 0.5172
Epoch 7/20 - Train Loss: 0.6359, Accuracy: 0.6486, Test Loss: 0.6843, Accuracy: 0.5732, Test F1: 0.5622
Epoch 8/20 - Train Loss: 0.6272, Accuracy: 0.6580, Test Loss: 0.7278, Accuracy: 0.5225, Test F1: 0.4663
Epoch 9/20 - Train Loss: 0.6191, Accuracy: 0.6620, Test Loss: 0.7526, Accuracy: 0.5274, Test F1: 0.4556
Epoch 10/20 - Train Loss: 0.6078, Accuracy: 0.